<a href="https://colab.research.google.com/github/jcdevaney/pyAMPACTtutorials/blob/main/test-04-pyAMPACT_symbolic_Overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>pyAMPACT Symbolic Processing Overview</h1>


In [1]:
# from repo
!git clone https://github.com/jcdevaney/pyAMPACTtutorials.git
!git clone https://github.com/jcdevaney/pyAMPACT.git
%cd /content/pyAMPACT/pyampact/
from IPython.utils import io
print('Importing libraries...')
with io.capture_output() as captured:
    !pip install --upgrade pandas

# from package
#!git clone https://github.com/jcdevaney/pyAMPACTtutorials.git
#from IPython.utils import io
#print('Importing libraries...')
#with io.capture_output() as captured:
#    !pip install --upgrade pandas
#    !pip install -i https://test.pypi.org/simple/ --no-deps pyampact==0.4.0
#    import pyampact

Cloning into 'pyAMPACTtutorials'...
remote: Enumerating objects: 251, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 251 (delta 51), reused 17 (delta 14), pack-reused 177
Receiving objects: 100% (251/251), 17.35 MiB | 16.68 MiB/s, done.
Resolving deltas: 100% (112/112), done.
Cloning into 'pyAMPACT'...
remote: Enumerating objects: 1244, done.
remote: Counting objects: 100% (241/241), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 1244 (delta 162), reused 184 (delta 126), pack-reused 1003
Receiving objects: 100% (1244/1244), 22.21 MiB | 9.67 MiB/s, done.
Resolving deltas: 100% (733/733), done.
/content/pyAMPACT/pyampact
Importing libraries...


You can import a piece with a filepath or a url. Any symbolic notation filetype that music21 imports is allowed and this includes all the major ones like .mei, .xml, .krn, .midi, etc. Let's import one from the test_files directory in the repo.

In [3]:
# script.py needs to be uploaded to /content/pyAMPACT/pyampact/
from script import Score
piece = Score('/content/pyAMPACTtutorials/test_files/Mozart_K179_seg.krn')
print(f'-> Successfully imported {piece.metadata["title"]} by {piece.metadata["composer"]}.\n')

-> Successfully imported Zwolf Variationen in C uber ein Menuett von Johann Christian Fischer by Mozart.



/content/pyAMPACT/pyampact/script.py:346: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dur = df.applymap(lambda cell: round(float(cell.quarterLength), 5), na_action='ignore')
/content/pyAMPACT/pyampact/script.py:662: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  priority = self._parts().applymap(lambda cell: cell.priority, na_action='ignore').ffill(axis=1).iloc[:, -1].astype('Int16')


Run this to see all the public methods and properties available on score objects:

In [4]:
print(piece.public)

cdata          <class 'method'>
chords         <class 'method'>
durations      <class 'method'>
dynamics       <class 'method'>
fileExtension  <class 'str'>
fileName       <class 'str'>
fromJSON       <class 'method'>
functions      <class 'method'>
harm           <class 'method'>
harmKeys       <class 'method'>
insertAudioAnalysis<class 'method'>
jsonCDATA      <class 'method'>
kernNotes      <class 'method'>
lyrics         <class 'method'>
mask           <class 'method'>
metadata       <class 'dict'>
midiPitches    <class 'method'>
nmats          <class 'method'>
notes          <class 'method'>
partNames      <class 'list'>
path           <class 'str'>
pianoRoll      <class 'method'>
sampled        <class 'method'>
score          <class 'music21.stream.base.Score'>
show           <class 'method'>
toKern         <class 'method'>
xmlIDs         <class 'method'>


Most of these are methods you can call on the Score object to get back a pandas dataframe (df) containing one type of data. For example, `.durations()` returns a df of the durations of all the notes and rests in the piece, as shown in the cell below. The columns are the names of the different parts in the piece according to the score encoding. We can see that there are four parts, shown as columns. Musical time serves as the index (the left-most numbers) for the rows. They are notated in music21 offsets, which indicate the number of quarter notes since the beginning of the piece. So 0 is the beginning of the piece and the second row of the table (at index 8.0) contains an event in each of the first two columns. The last two columns do not have new events at this moment in the piece so they get filled with placeholder NaNs.

In [5]:
piece.durations()

/content/pyAMPACT/pyampact/script.py:754: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  self._analyses['_m21ObjectsNoTies'] = self._parts(multi_index=True).applymap(self._removeTied).dropna(how='all')
/content/pyAMPACT/pyampact/script.py:848: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  res = m21objs.applymap(lambda nrc: nrc.quarterLength, na_action='ignore').astype(float).round(5)


,Part-1,Part-2
0.00000,2.00000,1.0
1.00000,NaN,1.0
2.00000,0.25000,1.0
2.25000,0.25000,NaN
2.50000,0.25000,NaN
2.75000,0.25000,NaN
3.00000,0.50000,1.0
3.50000,0.50000,NaN
4.00000,1.50000,1.0
5.00000,NaN,1.0


The table above gave us the durations of note and rest events, but how do we know if they're notes or rests? We can look at the `.midiPitches()` table to see what notes or rests those durations correspond to. In midi, notes are notated chromatically from 0 to 127 inclusive, where 60 is middle C (C4), 61 is C#/Db, 72 is C5, etc. Since there is no representation of rests in midi, -1s are used for rests. If you want more standard pitch names, you can use `.notes()` instead.

In [6]:
piece.midiPitches()

/content/pyAMPACT/pyampact/script.py:885: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  midiPitches = self._m21ObjectsNoTies().applymap(lambda nr: -1 if nr.isRest else nr.pitch.midi, na_action='ignore')


,Part-1,Part-2
0.00000,79.0,48.0
1.00000,NaN,52.0
2.00000,77.0,48.0
2.25000,76.0,NaN
2.50000,74.0,NaN
2.75000,72.0,NaN
3.00000,79.0,50.0
3.50000,77.0,NaN
4.00000,74.0,47.0
5.00000,NaN,43.0


A "piano roll" representation of the piece is also available as a 128-column table. Each row represents a midi pitch (0-127 inclusive) and the columns correspond to musical time in the piece in the same way as the rows do in the tables above. Rests are not represented here and you can no longer tell which voice is performing any given note. Since there are too many columns to show on screen, pandas prints a summary of the columns showing the first few and the last few, with a column of ellipses in the middle.

In [7]:
piece.pianoRoll()

/content/pyAMPACT/pyampact/script.py:1138: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pianoRoll.fillna(0, inplace=True)


,0.00000,1.00000,2.00000,2.25000,2.50000,2.75000,3.00000,3.50000,4.00000,5.00000,5.50000,6.00000,6.33333,6.50000,6.66667,7.00000,8.00000,9.00000,10.00000,11.00000
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
124,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
125,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
126,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Extending on the pianoRoll, the `.sampled()` method samples pianoRoll df at regular time intervals. It assumes the beat to be the quarter note and takes `bpm` and `obs` parameters to determine how often to sample the pianoRoll. The equation for the regular time intervals is (60/bpm)/obs. With the default values of 60 and 20 for `bpm` and `obs` respectively, this results in 20 observations per quarter note.

In [8]:
print(piece.sampled.__doc__)
piece.sampled()


    Sample the score according to the given beats per minute (bpm) and the 
    desired observations per second (obs).

    :param bpm: Integer, default 60. The beats per minute to use for sampling.
    :param obs: Integer, default 20. The desired observations per second.
    :return: A DataFrame representing the sampled score. Each row corresponds 
        to a MIDI pitch (0 to 127), and each column corresponds to a timepoint 
        in the sampled score. The values are 1 for a note onset and 0 otherwise.
    


,0.00,0.05,0.10,0.15,0.20,0.25,0.30,0.35,0.40,0.45,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.90,0.95,1.00,1.05,1.10,1.15,1.20,1.25,1.30,1.35,1.40,1.45,1.50,1.55,1.60,1.65,1.70,1.75,1.80,1.85,1.90,1.95,2.00,2.05,2.10,2.15,2.20,2.25,2.30,2.35,2.40,2.45,...,9.50,9.55,9.60,9.65,9.70,9.75,9.80,9.85,9.90,9.95,10.00,10.05,10.10,10.15,10.20,10.25,10.30,10.35,10.40,10.45,10.50,10.55,10.60,10.65,10.70,10.75,10.80,10.85,10.90,10.95,11.00,11.05,11.10,11.15,11.20,11.25,11.30,11.35,11.40,11.45,11.50,11.55,11.60,11.65,11.70,11.75,11.80,11.85,11.90,11.95
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
124,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
125,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
126,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


The `.mask()` is an important Score method. It is the python version of the matlab notes2mask function.

In [9]:
piece.mask()

/content/pyAMPACT/pyampact/script.py:1186: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mask = pd.DataFrame(index=range(num_rows), columns=sampled.columns).fillna(0)


,0.00,0.05,0.10,0.15,0.20,0.25,0.30,0.35,0.40,0.45,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.90,0.95,1.00,1.05,1.10,1.15,1.20,1.25,1.30,1.35,1.40,1.45,1.50,1.55,1.60,1.65,1.70,1.75,1.80,1.85,1.90,1.95,2.00,2.05,2.10,2.15,2.20,2.25,2.30,2.35,2.40,2.45,...,9.50,9.55,9.60,9.65,9.70,9.75,9.80,9.85,9.90,9.95,10.00,10.05,10.10,10.15,10.20,10.25,10.30,10.35,10.40,10.45,10.50,10.55,10.60,10.65,10.70,10.75,10.80,10.85,10.90,10.95,11.00,11.05,11.10,11.15,11.20,11.25,11.30,11.35,11.40,11.45,11.50,11.55,11.60,11.65,11.70,11.75,11.80,11.85,11.90,11.95
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
125,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
126,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
127,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Another python implementation of a matlab function that pyAMPACT offers is called `.nmats()`. Its dataframes correspond to the output of the `midi2nmat` run on each part in the score. Let's use the doc string of `.nmats()` to read more about it and then check out a sample.

In [10]:
print(piece.nmats.__doc__)
piece.nmats()


    Return a dictionary of DataFrames, one for each voice, with information 
    about the notes and rests in that voice.

    Each DataFrame has the following columns:
    
    MEASURE  ONSET  DURATION  PART  MIDI  ONSET_SEC  OFFSET_SEC
    
    In the MIDI column, notes are represented 
    with their MIDI pitch numbers (0 to 127), and rests are represented with -1s. 
    The ONSET_SEC and OFFSET_SEC columns are taken from the audio analysis from 
    the `json_path` file if one is given. The XML_IDs of each note or rest serve 
    as the index for this DataFrame. If `include_cdata` is True and a `json_path` 
    is provided, the cdata from the json file is included in the DataFrame.

    :param json_path: Optional path to a JSON file containing audio analysis data.
    :param include_cdata: Boolean, default False. If True and a `json_path` is 
        provided, the cdata from the json file is included in the DataFrame.
    :return: A dictionary of DataFrames, one for each voice.
  

/content/pyAMPACT/pyampact/script.py:848: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  res = m21objs.applymap(lambda nrc: nrc.quarterLength, na_action='ignore').astype(float).round(5)
/content/pyAMPACT/pyampact/script.py:885: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  midiPitches = self._m21ObjectsNoTies().applymap(lambda nr: -1 if nr.isRest else nr.pitch.midi, na_action='ignore')
/content/pyAMPACT/pyampact/script.py:563: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = self._parts(multi_index=True).applymap(lambda obj: str(obj.id), na_action='ignore')
/content/pyAMPACT/pyampact/script.py:1098: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always be

{'Part-1':                  MEASURE     ONSET  DURATION    PART  MIDI ONSET_SEC OFFSET_SEC
 XML_ID                                                                         
 138843347112896        1   0.00000   2.00000  Part-1  79.0       NaN        NaN
 138843347112992        1   2.00000   0.25000  Part-1  77.0       NaN        NaN
 138843347113088        1   2.25000   0.25000  Part-1  76.0       NaN        NaN
 138843347113184        1   2.50000   0.25000  Part-1  74.0       NaN        NaN
 138843347113280        1   2.75000   0.25000  Part-1  72.0       NaN        NaN
 138843347113472        2   3.00000   0.50000  Part-1  79.0       NaN        NaN
 138843347113568        2   3.50000   0.50000  Part-1  77.0       NaN        NaN
 138843347113664        2   4.00000   1.50000  Part-1  74.0       NaN        NaN
 138843347113760        2   5.50000   0.50000  Part-1  77.0       NaN        NaN
 138843347113952        3   6.00000   0.33333  Part-1  76.0       NaN        NaN
 138843347114144  

The alignment of the columns gets a little thrown off by the dictionary keys, so let's look at just the value for the part named "Part_2".

In [11]:
piece.nmats()['Part-2']

,MEASURE,ONSET,DURATION,PART,MIDI,ONSET_SEC,OFFSET_SEC
XML_ID,,,,,,,
138843347118080,1,0.0,1.0,Part-2,48.0,NaN,NaN
138843347118224,1,1.0,1.0,Part-2,52.0,NaN,NaN
138843347118368,1,2.0,1.0,Part-2,48.0,NaN,NaN
138843347118608,2,3.0,1.0,Part-2,50.0,NaN,NaN
138843347118752,2,4.0,1.0,Part-2,47.0,NaN,NaN
138843347118896,2,5.0,1.0,Part-2,43.0,NaN,NaN
138843347119136,3,6.0,0.5,Part-2,50.0,NaN,NaN
138843347119280,3,6.5,0.5,Part-2,53.0,NaN,NaN
138843347119424,3,7.0,1.0,Part-2,55.0,NaN,NaN


<h2>Experimental and in-progress features</h2>

Additional parsing of `.krn` scores is also available. If a **function spine is in the file, you can retrieve its scoretime-aligned values with the `.functions()` method. For non-kern files, or for kern files without a **function spine, this method returns an empty array. Let's look at the `cDataTest.krn` file which has a **functions spine.

In [13]:
piece = Score('/content/pyAMPACTtutorials/test_files/cDataTest.krn')
piece.functions()

/content/pyAMPACT/pyampact/script.py:346: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dur = df.applymap(lambda cell: round(float(cell.quarterLength), 5), na_action='ignore')
/content/pyAMPACT/pyampact/script.py:662: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  priority = self._parts().applymap(lambda cell: cell.priority, na_action='ignore').ffill(axis=1).iloc[:, -1].astype('Int16')


array(['T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T',
       'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'P',
       'P', 'P', 'P', 'D', 'D', 'T', 'T', 'T'], dtype=object)

If a .krn file has a **harm spine, we can similarly retrieve the analyzed harmonies with `piece.harm()`. We can also get the prevailing key analysis from the **harm spine (if there is one) with `piece.harmKeys()`. The default return type of functions, harmonies, and harmKeys is a numpy array, but let's use `output='series'` to keep this data as pandas series so that we can concatenate them into a table. This is a good opportunity to see how we can use some basic pandas methods to join our various tables on their musical time axis. We'll also forward-fill the observations, since the prevailing-key information is very sparse. This time let's use `.tail(10)` to look at the last 10 rows in the table.

In [14]:
import pandas as pd
functions = piece.functions(output='series')
harmonies = piece.harm(output='series')
keys = piece.harmKeys(output='series')
df = pd.concat([keys, harmonies, functions], axis=1)
df.columns = ['Key', 'Harmony', 'Function']
df.tail(10)

,Key,Harmony,Function
,,,
16.5,D,I,T
17.0,D,iib,P
18.0,D,iib,P
19.0,D,iib,P
19.5,D,iib,P
20.0,D,V,D
20.5,D,V,D
21.0,D,I,T
22.0,D,I,T


pyAMPACT also supports reading analyses encoded in a **cdata spine in a .krn file where each record is stored as a json object (similar to a python dictionary). Let's look at the first five rows of this data for the same piece using the pandas `.head()` method.

In [15]:
piece.cdata().head()

,dur,f0Vals,ppitch,jitter,vibratoDepth,vibratoRate,pwrVals,avgPwr,shimmer,specCent,specCentMean,specSlope,meanSpecSlope,specFlux,meanspecFlux,specFlat,meanspecFlat
,,,,,,,,,,,,,,,,,
0.0,0.082721,"[414.45386767783316, 414.84188220077607, 417.1...",419.674139,0.989929,2.888633,14.355469,"[8.416609281735337, 9.144024591233709, 8.67927...",7.8353,6.884973,"[1546.7365173078974, 1617.2715395815958, 1605....",1370.159453,"[-1.776842897401089e-05, -2.088842207601961e-0...",-0.000011,"[0, 0.0005836056244841356, 0.00063331141294428...",0.000319,"[2.0897882834210336e-16, 1.583077320977022e-16...",1.978027e-13
1.0,0.082721,"[414.45386767783316, 414.84188220077607, 417.1...",419.674139,0.989929,2.888633,14.355469,"[8.416609281735337, 9.144024591233709, 8.67927...",7.8353,6.884973,"[1546.7365173078974, 1617.2715395815958, 1605....",1370.159453,"[-1.776842897401089e-05, -2.088842207601961e-0...",-0.000011,"[0, 0.0005836056244841356, 0.00063331141294428...",0.000319,"[2.0897882834210336e-16, 1.583077320977022e-16...",1.978027e-13
2.0,0.082721,"[414.45386767783316, 414.84188220077607, 417.1...",419.674139,0.989929,2.888633,14.355469,"[8.416609281735337, 9.144024591233709, 8.67927...",7.8353,6.884973,"[1546.7365173078974, 1617.2715395815958, 1605....",1370.159453,"[-1.776842897401089e-05, -2.088842207601961e-0...",-0.000011,"[0, 0.0005836056244841356, 0.00063331141294428...",0.000319,"[2.0897882834210336e-16, 1.583077320977022e-16...",1.978027e-13
2.5,0.082721,"[414.45386767783316, 414.84188220077607, 417.1...",419.674139,0.989929,2.888633,14.355469,"[8.416609281735337, 9.144024591233709, 8.67927...",7.8353,6.884973,"[1546.7365173078974, 1617.2715395815958, 1605....",1370.159453,"[-1.776842897401089e-05, -2.088842207601961e-0...",-0.000011,"[0, 0.0005836056244841356, 0.00063331141294428...",0.000319,"[2.0897882834210336e-16, 1.583077320977022e-16...",1.978027e-13
3.0,0.082721,"[414.45386767783316, 414.84188220077607, 417.1...",419.674139,0.989929,2.888633,14.355469,"[8.416609281735337, 9.144024591233709, 8.67927...",7.8353,6.884973,"[1546.7365173078974, 1617.2715395815958, 1605....",1370.159453,"[-1.776842897401089e-05, -2.088842207601961e-0...",-0.000011,"[0, 0.0005836056244841356, 0.00063331141294428...",0.000319,"[2.0897882834210336e-16, 1.583077320977022e-16...",1.978027e-13


As with the `.functions()`, `.chords()`, `.harm()`, and `.harmKeys()` methods, `.cdata()` can also take `snap_to`, `filler`, and `output` arguments. See the doc string of any of these methods to learn more:

In [16]:
print(piece.functions.__doc__)


  The methods .harmKeys, .harm, .functions, .chords, and .cdata all work in 
  the following way. They get the desired analysis from the relevant spine if 
  this piece is a kern file and has that spine. The default is for the results 
  to be returned as a 1-d array, but you can set `output='series'` for a pandas 
  series instead. If you want to align these results so that they match the 
  columnar (time) axis of the pianoRoll, sampled, or mask results, you can pass 
  the pianoRoll or mask that you want to align to as the `snap_to` parameter.

  The `sampled` and `mask` dfs often have more observations than the spine 
  contents, so you may want to fill in these new empty slots somehow. The kern 
  format uses '.' as a filler token so you can pass this as the `filler` 
  parameter to fill all the new empty slots with this as well. If you choose 
  some other value, say `filler='_'`, then in addition to filling in the empty 
  slots with underscores, this will also replace the kern

<h2>But how do you get your data into the score in the first place?!</h2>

Say you have a score in symbolic notation and a *seperate* json file containing audio analysis of a performance of that piece. It can be very useful to insert that analysis directly into the score. pyAMPACT lets you import the data, align it with the score, and output it as a `kern` or `mei` score. That also means that pyAMPACT lets you **output** a valid kern or mei version of any of the file types that music21 **imports**. Since music21 imports a lot of file types, that means pyAMPACT is an almost universal X-to-kern and X-to-mei converter. Let's look at this starting with importing a json file of analysis.

<h3>Import a json file</h3>

Provided your json file is in the right format, run `fromJSON(path_to_json_file)` passing in the path to the json file. For demonstration purposes, there's a small json file with a small sample of dummy data in the `test_files` directory called cdataCompact.json. The outermost keys should be in seconds with decimal places allowed (i.e. 123.5 for two minutes and three and a half seconds). We use these as the index values for the rows. The 2nd-level keys (of the nested objects) get used as the column names of the resultant table. Let's use an excerpt from a Mozart piece in the TAVERN repository for these next few examples, and see that json file as a table.

In [19]:
piece = Score(score_path='/content/pyAMPACTtutorials/test_files/Mozart_K179_seg.krn')
fromJSON('/content/pyAMPACTtutorials/test_files/cdataCompact.json')

/content/pyAMPACT/pyampact/script.py:346: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dur = df.applymap(lambda cell: round(float(cell.quarterLength), 5), na_action='ignore')
/content/pyAMPACT/pyampact/script.py:662: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  priority = self._parts().applymap(lambda cell: cell.priority, na_action='ignore').ffill(axis=1).iloc[:, -1].astype('Int16')


NameError: name 'fromJSON' is not defined

<h3>Export a file to kern</h3>

Now lets export this `.mid` file to a valid `.krn` file. You can pass a `path_name` parameter which will be the name of the file. The file will get saved in the `./output` folder. If you don't add a `.krn` extension to the end of the file name, it will get added automatically. If you don't pass a `path_name` the function will return the file as a string rather than saving the file. Be aware that this will overwrite an existing file at the same path if there is one.

In [ ]:
piece.toKern('/content/pyAMPACTtutorials/output_files/Mozart')

<h3>Export a file to kern with the json analysis included</h3>

That's great, but what about inserting the json analysis data into the score? `.toKern` takes an optional second parameter called `data`. You pass it the path of the json file and pyAMPACT takes care of the rest.

In [ ]:
piece.toKern(path_name='/content/pyAMPACTtutorials/Mozart_with_json_analysis', data='/content/pyAMPACTtutorials/test_files/cdataCompact.json')

AttributeError: 'str' object has no attribute 'name'